# RNN Module

rnn 버리고 lstm으로 먼저 구현해봅니다 <br />
input : FormattedReview List <br />
output : numpy array <br />

FormattedReview
- context              : Numpy array
- context_bayes        : Tuple List
- comp_similarity      : float
- rate                 : float
- reiteration_context  : float
- reiteration_repeat   : float
- post_time            : float
- post_vip             : float
- id                   : float

- index                : int
- label                : bool

sample code from <br />
https://discuss.pytorch.org/t/example-of-many-to-one-lstm/1728/2

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

hidden_size = 10  # hyperparameter
num_layers = 3    # hyperparameter
p_dropout = 0.05  # hyperparameter

### sort_batch from
https://discuss.pytorch.org/t/about-the-variable-length-input-in-rnn-scenario/345/22

In [5]:
class RNN_model(nn.Module):
    def __init__(self, input_size):
        super(RNN_model, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout = p_dropout)
        
    def forward(self, embeddings, lengths, hidden=None):
        
        packed = nn.utils.rnn.pack_padded_sequence(embeddings, lengths.tolist(), batch_first=True)
        output, hidden = self.rnn(packed, hidden)
        output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        
        idx = (lengths-1).view(-1,1).expand(output.size(0), output.size(2)).unsqueeze(1)
        
        #print(output)
        decoded = output.gather(1, Variable(idx)).squeeze()
        
        return decoded